In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from mqt.predictor import rl, ml
df = pd.read_csv(ml.helper.get_path_trained_model() / 'res.csv', sep=',')

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 30

In [ ]:
df

In [ ]:
fidelity_cols = df.columns[df.columns.str.endswith('_fidelity')]

# Find the column name with the maximum value for each row
max_cols = df[fidelity_cols].idxmax(axis=1)

# Convert the result to a list
max_cols_list = max_cols.tolist()

print(set(max_cols_list))

## Add best datapoints from other compilation flows

In [ ]:
## Add third highest data point, mean, best and worst

In [ ]:
def calculate_x_highest(row, column_name, index):
    columns = [col for col in row.index if col.endswith(column_name) and not "MQT" in col]
    values = sorted([value for col, value in row[columns].items() if value != -1], reverse=True)
    return values[index-1] if len(values) >= 3 else None

def calculate_mean(row, column_name):
    columns = [col for col in row.index if col.endswith(column_name) and not "MQT" in col]
    values = [value for col, value in row[columns].items() if value != -1]
    return np.mean(values) if values else None

df['fidelity_second_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=2, axis=1)
df['fidelity_third_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=3, axis=1)
df['fidelity_mean'] = df.apply(calculate_mean, column_name="_fidelity",axis=1)


df['critical_depth_third_highest'] = df.apply(calculate_x_highest, column_name="_critical_depth", index=3,axis=1)
df['critical_depth_mean'] = df.apply(calculate_mean, column_name="_critical_depth",axis=1)

In [ ]:
df['fidelity_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['fidelity_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].max(), axis=1)
df['critical_depth_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['critical_depth_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].max(), axis=1)


In [ ]:
predictor = ml.Predictor()
(X_train,
X_test,
y_train,
y_test,
indices_train,
indices_test,
names_list,
scores_list) = predictor.get_prepared_training_data(figure_of_merit="fidelity", save_non_zero_indices=True)
test_benchmarks_fidelity = [names_list[index_test] for index_test in indices_test]
df_filtered_fidelity = df[df["file_path"].isin(test_benchmarks_fidelity)]

In [ ]:
plot_kind = "line"
df = df_filtered_fidelity.sort_values(by=['MQTPredictor_fidelity_fidelity'])
ax = df.plot(x="file_path", y="fidelity_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(30,10))
df.plot(x="file_path", y="MQTPredictor_fidelity_fidelity", label="MQT Predictor", kind=plot_kind, rot=90, ax=ax, color="blue")
df.plot(x="file_path", y="fidelity_third_highest", kind=plot_kind, rot=90, ax=ax, color="purple", label="3rd Best")
df.plot(x="file_path", y="fidelity_mean", kind=plot_kind, rot=90, ax=ax, color="orange", label="Mean")
df.plot(x="file_path", y="fidelity_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df.file_path), 1)),
    ["" for i in range(len(df.file_path))],
    fontsize=16,
)
plt.xlabel("Benchmarks")
plt.ylabel("Expected Fidelity")

plt.savefig("results/fidelity_plot.pdf", bbox_inches='tight')

In [ ]:
predictor = ml.Predictor()
(X_train,
X_test,
y_train,
y_test,
indices_train,
indices_test,
names_list,
scores_list) = predictor.get_prepared_training_data(figure_of_merit="critical_depth", save_non_zero_indices=True)
test_benchmarks_critical_depth = [names_list[index_test] for index_test in indices_test]
df_filtered_critical_depth = df[df["file_path"].isin(test_benchmarks_critical_depth)]

In [ ]:
plot_kind = "line"
df = df_filtered_critical_depth.sort_values(by=['MQTPredictor_critical_depth_critical_depth'])
ax = df.plot(x="file_path", y="critical_depth_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(30,10))
df.plot(x="file_path", y="MQTPredictor_critical_depth_critical_depth", kind=plot_kind, rot=90, ax=ax, color="blue", label="MQT Predictor")
df.plot(x="file_path", y="critical_depth_mean", kind=plot_kind, rot=90, ax=ax, color="orange", label="Mean")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path);

plt.xticks(
    list(range(0, len(df.file_path), 1)),
    ["" for i in range(len(df.file_path))],
    fontsize=16,
)
plt.xlabel("Benchmarks")
plt.ylabel("Critical Depth")

plt.savefig("results/critical_depth_plot.pdf", bbox_inches='tight')

## Normalize

In [ ]:
# for index, row in df.iterrows():
#     max_fidelity = max(row[col] for col in df.columns if col.endswith('_fidelity') and "MQT" not in col)
#     for col in df.columns:
#         if col.endswith('_fidelity'):
#             df.at[index, col] /= max_fidelity

# for index, row in df.iterrows():
#     max_crit_depth = max(row[col] for col in df.columns if col.endswith('_critical_depth') and "MQT" not in col)
#     if max_crit_depth == 0:
#         continue
#     for col in df.columns:
#         if col.endswith('_critical_depth'):
#             df.at[index, col] /= max_crit_depth

# New Evaluation Scheme

In [ ]:
list_cols_fid = [x for x in df.columns if x.endswith("fidelity")]

In [ ]:
cmap = plt.get_cmap("viridis", len(list_cols_fid))
custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]

# New Rankings

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
    plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
    plt.xlim(bins[0], bins[-1])

In [ ]:
df['Rank_MQT_fidelity'] = df.apply(lambda row: sum(1 for col in df.columns if col.endswith('_fidelity') and not "MQT" in col and row['MQTPredictor_fidelity_fidelity'] >= row[col]), axis=1)
df['Rank_MQT_fidelity'] = len([col for col in df.columns if col.endswith('_fidelity') and not "MQT" in col]) - df.Rank_MQT_fidelity
plt.hist(df.Rank_MQT_fidelity.values, bins=range(0,15,1), align="left")
plt.xticks(range(0,14,1));

In [ ]:
len(df[df['Rank_MQT_fidelity']==0])

In [ ]:
df['Rank_MQT_critical_depth'] = df.apply(lambda row: sum(1 for col in df.columns if col.endswith('_critical_depth') and not "MQT" in col and row['MQTPredictor_critical_depth_critical_depth'] >= row[col]), axis=1)
df['Rank_MQT_critical_depth'] = len([col for col in df.columns if col.endswith('_critical_depth') and not "MQT" in col]) - df.Rank_MQT_critical_depth
plt.hist(df.Rank_MQT_critical_depth.values, bins=range(0,15,1), align="left")
plt.xticks(range(0,14,1));

In [ ]:
len(df[df['Rank_MQT_critical_depth']==0])

# Expected Fidelity

# 2x2 Matrix With Mean Results and Optimization Criterion Comparison

In [ ]:
MQT_fidelity = [df["MQTPredictor_fidelity_fidelity"].mean(),  df["MQTPredictor_fidelity_critical_depth"].mean()]
MQT_critical_depth = [df["MQTPredictor_critical_depth_fidelity"].mean(), df["MQTPredictor_critical_depth_critical_depth"].mean()]

In [ ]:
print(np.round(MQT_fidelity,2))
print(np.round(MQT_critical_depth,2))

# GHZ Evaluation

In [ ]:
df = pd.read_csv(ml.helper.get_path_trained_model() / 'res_GHZ.csv', sep=',')
plt.rcParams["font.size"] = 50

In [ ]:
df['fidelity_second_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=2, axis=1)
df['fidelity_third_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=3, axis=1)
df['fidelity_mean'] = df.apply(calculate_mean, column_name="_fidelity",axis=1)


df['critical_depth_third_highest'] = df.apply(calculate_x_highest, column_name="_critical_depth", index=3,axis=1)
df['critical_depth_mean'] = df.apply(calculate_mean, column_name="_critical_depth",axis=1)

df['fidelity_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['fidelity_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].max(), axis=1)
df['critical_depth_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['critical_depth_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].max(), axis=1)


In [ ]:
plot_kind = "line"
df = df.sort_values(by=['num_qubits'])
ax = df.plot(x="file_path", y="fidelity_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(20,10))
df.plot(x="file_path", y="MQTPredictor_fidelity_fidelity", label="MQT Predictor", kind=plot_kind, rot=90, ax=ax, color="blue")
df.plot(x="file_path", y="fidelity_third_highest", kind=plot_kind, rot=90, ax=ax, color="purple", label="3rd Best")
df.plot(x="file_path", y="fidelity_mean", kind=plot_kind, rot=90, ax=ax, color="orange", label="Mean")
df.plot(x="file_path", y="fidelity_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Benchmarks")
plt.ylabel("Critical Depth")


plt.savefig("results/fidelity_ghz_plot.pdf", bbox_inches='tight')

In [ ]:
plot_kind = "line"
df = df.sort_values(by=['num_qubits'])
ax = df.plot(x="file_path", y="critical_depth_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(20,10))
df.plot(x="file_path", y="MQTPredictor_critical_depth_critical_depth", kind=plot_kind, rot=90, ax=ax, color="blue", label="MQT Predictor")
df.plot(x="file_path", y="critical_depth_mean", kind=plot_kind, rot=90, ax=ax, color="orange", label="Mean")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path);

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Benchmarks")
plt.ylabel("Critical Depth")

plt.savefig("results/critical_depth_ghz_plot.pdf", bbox_inches='tight')